In [20]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-25 03:16:24--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-25 03:16:24 (60.3 MB/s) - ‘minsearch.py.1’ saved [4073/4073]



In [21]:
import minsearch
import json

In [22]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [23]:
index = minsearch.Index(
    text_fields = ["question","text","section"],
    keyword_fields=["course"]
)

In [24]:
# SELECT * WHERE course = 'data-engineering-zoomcamp';

In [25]:
q = 'the course has already started, can I still enrolled?'

In [26]:
index.fit(documents)

In [27]:
# boot the importance of quesiton field
# and section field less important
boost = {"question":3.0, 'section':0.5}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [28]:
q

'the course has already started, can I still enrolled?'

In [29]:
import os
import requests
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Get the key from the environment
groq_api_key = os.getenv("GROQ_API_KEY")

# API request setup
url = "https://api.groq.com/openai/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {groq_api_key}"
}

data = {
    "model": "meta-llama/llama-4-scout-17b-16e-instruct",
    "messages": [
        {"role": "user", "content": q}
    ]
}

# Send the request
response = requests.post(url, headers=headers, json=data)

# Print the result
result = response.json()
print(result["choices"][0]["message"]["content"])

Unfortunately, I don't have enough information to provide a definitive answer, as it depends on the specific course and institution you're interested in. However, I can offer some general guidance.

**Possible scenarios:**

1. **Courses with rolling admissions**: Some courses or programs may have rolling admissions, which means you can still enroll even after the course has started. This is more common in online courses or programs with a flexible schedule.
2. **Courses with a late start date**: Some courses may have a late start date or a staggered start schedule, which could still allow you to enroll.
3. **Courses with available spots**: If there are still available spots in the course, the instructor or institution may allow you to enroll, even if the course has already started.

**What to do:**

1. **Check the course website or syllabus**: Look for information on the course website, syllabus, or contact the instructor to see if there are any specific enrollment deadlines or restric

In [30]:
prompt_template = """
You're a course teaching assistent. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [31]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n

SyntaxError: unterminated f-string literal (detected at line 3) (3240599338.py, line 3)

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

In [ ]:
print(prompt)

In [ ]:
data = {
    "model": "meta-llama/llama-4-scout-17b-16e-instruct",
    "messages": [
        {"role": "user", "content": prompt}
    ]
}

# Send the request
response = requests.post(url, headers=headers, json=data)

# Print the result
result = response.json()
print(result["choices"][0]["message"]["content"])

In [ ]:
def search(query):
    # boot the importance of quesiton field
    # and section field less important
    boost = {"question":3.0, 'section':0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [84]:
def build_prompt(query,search_results):
    prompt_template = """
    You're a course teaching assistent. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\\nquestion: {doc['question']}\\nanswer: {doc['text']}\\n\\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [85]:
def llm(prompt):
    data = {
    "model": "meta-llama/llama-4-scout-17b-16e-instruct",
    "messages": [
        {"role": "user", "content": prompt}
    ]}
    
    # Send the request
    response = requests.post(url, headers=headers, json=data)
    result = response.json()
    # print(result["choices"][0]["message"]["content"])
    return result["choices"][0]["message"]["content"]

In [86]:
query = 'how do i run kafka?'

In [87]:
# rag(query)
rag('the course has already started, can I still enroll?')

/tmp/ipykernel_104302/58569821.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


"Yes, you can still enroll in the course even though it has already started. You are still eligible to submit homework assignments. However, be aware that there will be deadlines for turning in the final projects, so plan accordingly and don't leave everything for the last minute."

### ElasticSearch

In [88]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [89]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch(
    "http://localhost:9200"
)
es_client.info()

ObjectApiResponse({'name': '21ab76c9e656', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'uOUPTcbFQDeCIyUCb60eiQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [90]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# es_client.indices.create(index=index_name, body=index_settings)
if not es_client.indices.exists(index=index_name):
    es_client.indices.create(
        index=index_name,
        settings=index_settings["settings"],
        mappings=index_settings["mappings"]
    )
else:
    print(f"Index '{index_name}' already exists.")

Index 'course-questions' already exists.


In [91]:
settings = es_client.indices.get_settings(index=index_name)
print(settings)

{'course-questions': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'course-questions', 'creation_date': '1750821331524', 'number_of_replicas': '0', 'uuid': 'I-VHYpbOT8iD7u1Of3v3iw', 'version': {'created': '8040399'}}}}}


In [92]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [93]:
from tqdm.auto import tqdm

In [94]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [96]:
query = "i just discovered this course. can i still join?"

In [97]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [98]:
def rag(query):
    search_results=elastic_search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [99]:
rag(query)

/tmp/ipykernel_104302/58569821.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


"Yes, you can still join the course. Even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."